In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Nodes & Edges

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

model = ChatOpenAI(temperature=0)

def add_one(input: list[HumanMessage]):
    input[0].content = input[0].content + "a"
    return input


graph = MessageGraph()

graph.add_node("branch_a", add_one)
graph.add_edge("branch_a", "branch_b")
graph.add_edge("branch_a", "branch_c")

graph.add_node("branch_b", add_one)
graph.add_node("branch_c", add_one)

graph.add_edge("branch_b", "final_node")
graph.add_edge("branch_c", "final_node")

graph.add_node("final_node", add_one)
graph.add_edge("final_node", "end_code")

graph.add_node("end_code", add_one)
graph.add_edge("end_code", END)

graph.set_entry_point("branch_a")

runnable = graph.compile()


In [ ]:
from IPython.display import Image, display

display(Image(runnable.get_graph().draw_mermaid_png()))

In [ ]:
runnable.invoke("a")

### Conditional Edges

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

model = ChatOpenAI(temperature=0)


def entry(input: list[HumanMessage]):
    return input

def work_with_b(input: list[HumanMessage]):
    print("Using branch B")
    return "b"

def work_with_c(input: list[HumanMessage]):
    print("Using branch C")
    return "c"

def router(input: list[HumanMessage]):
    if "use_b" in input[0].content:
        return "branch_b"
    else:
        return "branch_c"

graph = MessageGraph()

graph.add_node("branch_a", entry)
graph.add_node("branch_b", work_with_b)
graph.add_node("branch_c", work_with_c)

graph.add_conditional_edges(
    "branch_a",
    router,
    {"branch_b": "branch_b", "branch_c": "branch_c"}
)
graph.add_edge("branch_b", END)
graph.add_edge("branch_c", END)

graph.set_entry_point("branch_a")

runnable = graph.compile()

In [ ]:
runnable = graph.compile()

In [ ]:
display(Image(runnable.get_graph().draw_mermaid_png()))

In [ ]:
runnable.invoke("hello")

In [ ]:
runnable.invoke("I want to use_b")

### Cycles

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

model = ChatOpenAI(temperature=0)

def entry(input: list[HumanMessage]):
    print("Agent processing:", input[-1].content)
    if "end" in input[-1].content:
        return ["end"]
    return ["continue"]

def action(input: list[HumanMessage]):
    print("Action taken:", [msg.content for msg in input])
    if len(input) > 5:
        input.append(HumanMessage(content="end"))
    else:
        input.append(HumanMessage(content="continue"))
    return input

def should_continue(input: list):
    last_message = input[-1]
    if "end" in last_message.content:
        return "__end__"
    return "action"


graph = MessageGraph()

graph.add_node("agent", entry)
graph.add_node("action", action)

graph.add_conditional_edges(
    "agent",
    should_continue,
    {"action": "action", "__end__": END}
)
graph.add_edge("action", "agent")

graph.set_entry_point("agent")

runnable = graph.compile()


In [ ]:
display(Image(runnable.get_graph().draw_mermaid_png()))

In [ ]:
runnable.invoke("Hello")

### AgentState

In [ ]:
import random
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal

class AgentState(TypedDict):
    x: int
    y: int
    iteration: int

def increase_x_and_y(state: AgentState) -> AgentState:
    state['x'] += random.randint(0, 10)
    state['y'] += random.randint(0, 10)
    state['iteration'] += 1
    return state

def decrease_x_and_y(state: AgentState) -> AgentState:
    state['x'] -= random.randint(0, 10)
    state['y'] -= random.randint(0, 10)
    state['iteration'] += 1
    return state

def should_continue(state: AgentState) -> Literal["branch_a", "branch_b", "__end__"]:
    if state['iteration'] >= 10:
        return "__end__"
    elif state['x'] % 2 == 0:
        return "branch_a"
    else:
        return "branch_b"

workflow = StateGraph(AgentState)

workflow.add_node("branch_a", increase_x_and_y)
workflow.add_node("branch_b", decrease_x_and_y)

workflow.set_entry_point("branch_a")

workflow.add_conditional_edges(
    "branch_a",
    should_continue,
    {"branch_a": "branch_a", "branch_b": "branch_b", "__end__": END}
)
workflow.add_conditional_edges(
    "branch_b",
    should_continue,
    {"branch_a": "branch_a", "branch_b": "branch_b", "__end__": END}
)

app = workflow.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
initial_state = {"x": 10, "y": 10, "iteration": 0}
result = app.invoke(initial_state)
print(result)